In [1]:
!pip install transformers sentencepiece --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.0 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [3]:
model_checkpoint = 'cointegrated/rubert-tiny-toxicity'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()

In [34]:
def text2toxicity(text, aggregate=True):

    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(model.device)
        proba = torch.sigmoid(model(**inputs).logits).cpu().numpy()
    if isinstance(text, str):
        proba = proba[0]
    if aggregate:
        return 1 - proba.T[0] * (1 - proba.T[-1])
    return proba

# phrases are taken from media and talks

print(text2toxicity('There are too many blacks in Africa', True), '(1)') #  Afro-Americans
print(text2toxicity('We urgently need to hire a janitor', True), '(2)') #  environmental hygienist
print(text2toxicity('You eat like a pig, that is why you are so fat', False), '(3)') #  big-boned
print(text2toxicity('He called me a faggot when we met in Boston', True), '(4)', '\n') #  gay

print(text2toxicity('На рынке много чурок', True), '(5)')
print(text2toxicity('Эту страну населяют в основном узкоглазые', True), '(6)')
print(text2toxicity('Брат царя отправился охотиться на японских макак', True), '(7)')
print(text2toxicity('Здесь столько шлюх, что на всех хватит', True), '(8)')





0.43765847466693586 (1)
0.511504506565263 (2)
[9.9647754e-01 4.0657157e-03 7.0288812e-04 1.8259471e-04 3.8973656e-01] (3)
0.05523845310665365 (4) 

0.6048623092688921 (5)
0.8900869788768944 (6)
0.6723684151462379 (7)
0.9985226547872563 (8)
